# Evaluation Methods Demonstration

This notebook demonstrates the various evaluation methods available in the IDP library for comparing expected values with actual extraction results. It covers:

1. All evaluation methods with both match and no-match scenarios
2. Threshold testing for applicable methods
3. Edge cases:
   - Attribute not found in actual results
   - Attribute not found in expected results
   - Attribute not found in either actual or expected results

In [1]:
# First uninstall existing package (to ensure we get the latest version)
%pip uninstall -y idp_common

# Install the IDP common package with all components in development mode
%pip install -q -e "../lib/idp_common_pkg[all]"

Found existing installation: idp_common 0.2.19
Uninstalling idp_common-0.2.19:
  Successfully uninstalled idp_common-0.2.19
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import necessary libraries
import sys
import os
import json
from typing import Dict, Any, List, Tuple, Optional
import logging

# Add parent directory to path to import the library
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Import IDP libraries
from idp_common.evaluation.models import EvaluationMethod
from idp_common.evaluation.comparator import compare_values
from idp_common.evaluation.service import EvaluationService
from idp_common.models import Document, Section, Status

print("Libraries imported successfully")

Libraries imported successfully


## Part 1: Comparing Individual Values with Different Methods

We'll test each evaluation method with matching and non-matching examples.

In [3]:
def test_comparison(method: EvaluationMethod, expected: Any, actual: Any, 
                    threshold: float = 0.8, document_class: str = "TestDoc",
                    attr_name: str = "test_attr", attr_description: str = "Test attribute"):
    """Test a comparison method and print results."""
    
    print(f"\n{'-'*60}")
    print(f"Method: {method.name}")
    print(f"Expected: {expected}")
    print(f"Actual: {actual}")
    
    if method in [EvaluationMethod.FUZZY, EvaluationMethod.SEMANTIC]:
        print(f"Threshold: {threshold}")
    
    # Set up LLM config for the LLM method
    llm_config = None
    if method == EvaluationMethod.LLM:
        llm_config = {
            "model": "us.amazon.nova-lite-v1:0",
            "temperature": 0.0,
            "top_k": 250,
            "system_prompt": "You are an evaluator that helps determine if the predicted and expected values match for document attribute extraction.",
            "task_prompt": """I need to evaluate attribute extraction for a document of class: {DOCUMENT_CLASS}.

For the attribute named "{ATTRIBUTE_NAME}" described as "{ATTRIBUTE_DESCRIPTION}":
- Expected value: {EXPECTED_VALUE}
- Actual value: {ACTUAL_VALUE}

Do these values match in meaning, taking into account formatting differences, word order, abbreviations, and semantic equivalence?
Provide your assessment as a JSON with three fields:
- "match": boolean (true if they match, false if not)
- "score": number between 0 and 1 representing the confidence/similarity score
- "reason": brief explanation of your decision

Respond ONLY with the JSON and nothing else.  Here's the exact format:
{
  "match": true or false,
  "score": 0.0 to 1.0,
  "reason": "Your explanation here"
}
"""
        }
    
    # Perform the comparison
    matched, score, reason = compare_values(
        expected=expected,
        actual=actual,
        method=method,
        threshold=threshold,
        document_class=document_class,
        attr_name=attr_name,
        attr_description=attr_description,
        llm_config=llm_config
    )
    
    print(f"Matched: {matched}")
    print(f"Score: {score}")
    if reason:
        print(f"Reason: {reason}")
        
    return matched, score, reason

### Test 1: EXACT Method
Testing exact string matching with both match and non-match cases.

In [4]:
# EXACT method - Match
test_comparison(EvaluationMethod.EXACT, "Account #12345", "Account #12345")

# EXACT method - No match
test_comparison(EvaluationMethod.EXACT, "Account #12345", "Account #12346")

# EXACT method - Match with different casing and punctuation
test_comparison(EvaluationMethod.EXACT, "Account Number: 12345", "account number 12345")


------------------------------------------------------------
Method: EXACT
Expected: Account #12345
Actual: Account #12345
Matched: True
Score: 1.0

------------------------------------------------------------
Method: EXACT
Expected: Account #12345
Actual: Account #12346
Matched: False
Score: 0.0

------------------------------------------------------------
Method: EXACT
Expected: Account Number: 12345
Actual: account number 12345
Matched: True
Score: 1.0


(True, 1.0, None)

### Test 2: NUMERIC_EXACT Method
Testing numeric comparison with different formats.

In [5]:
# NUMERIC_EXACT method - Match
test_comparison(EvaluationMethod.NUMERIC_EXACT, "$1,250.00", 1250)

# NUMERIC_EXACT method - No match
test_comparison(EvaluationMethod.NUMERIC_EXACT, "$1,250.00", 1251)

# NUMERIC_EXACT method - Match with different formats
test_comparison(EvaluationMethod.NUMERIC_EXACT, "(1,250.00)", "-1250")


------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: $1,250.00
Actual: 1250
Matched: True
Score: 1.0

------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: $1,250.00
Actual: 1251
Matched: False
Score: 0.0

------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: (1,250.00)
Actual: -1250
Matched: False
Score: 0.0


(False, 0.0, None)

### Test 3: FUZZY Method
Testing fuzzy comparison with different thresholds.

In [6]:
# FUZZY method - High match
test_comparison(EvaluationMethod.FUZZY, "John A. Smith", "John Smith", threshold=0.8)

# FUZZY method - Medium match 
matched, score, _ = test_comparison(EvaluationMethod.FUZZY, "John A. Smith", "John Simpson", threshold=0.8)
print(f"With threshold=0.6: {score >= 0.6}")

# FUZZY method - Low match
test_comparison(EvaluationMethod.FUZZY, "John Alexander Smith", "Jane Marie Johnson", threshold=0.8)


------------------------------------------------------------
Method: FUZZY
Expected: John A. Smith
Actual: John Smith
Threshold: 0.8
Matched: True
Score: 0.8333333333333334

------------------------------------------------------------
Method: FUZZY
Expected: John A. Smith
Actual: John Simpson
Threshold: 0.8
Matched: False
Score: 0.41666666666666663
With threshold=0.6: False

------------------------------------------------------------
Method: FUZZY
Expected: John Alexander Smith
Actual: Jane Marie Johnson
Threshold: 0.8
Matched: False
Score: 0.15000000000000002


(False, 0.15000000000000002, None)

<cell_type>markdown</cell_type>### Test 4: HUNGARIAN Method
Testing list comparison using the Hungarian algorithm.

The Hungarian method is used for optimal matching between two lists when order doesn't matter. It automatically pairs items from each list for the maximum possible match score.

**New Feature:** The Hungarian method now supports a `comparator_type` configuration which can be set to:
- `EXACT`: Exact string matching (default)
- `FUZZY`: Fuzzy string matching with similarity scoring
- `NUMERIC`: Numeric comparison for handling different number formats

This allows for greater flexibility when comparing lists of different types of values.

In [7]:
# HUNGARIAN method - Full match with EXACT comparator (default)
expected_list = ["Deposit: $500", "Withdrawal: $150", "Transfer: $200"]
actual_list = ["Deposit: $500", "Transfer: $200", "Withdrawal: $150"]
test_comparison(EvaluationMethod.HUNGARIAN, expected_list, actual_list)

# HUNGARIAN method - Test with explicit comparator_type
print("\nDemonstrating the new comparator_type functionality:")
print("-" * 60)

# Test with EXACT comparator
expected_list = ["Deposit: $500", "Withdrawal: $150", "Transfer: $200"]
actual_list = ["Deposit: $500", "Withdrawal: $150", "Transfer: $210"]  # Small difference in last value
matched, score, reason = compare_values(
    expected=expected_list,
    actual=actual_list,
    method=EvaluationMethod.HUNGARIAN,
    comparator_type="EXACT"  # Using the new comparator_type parameter
)
print(f"HUNGARIAN with EXACT comparator:")
print(f"  Expected: {expected_list}")
print(f"  Actual: {actual_list}")
print(f"  Result: Matched={matched}, Score={score:.2f}")

# Test with FUZZY comparator
matched, score, reason = compare_values(
    expected=expected_list,
    actual=actual_list,
    method=EvaluationMethod.HUNGARIAN,
    comparator_type="FUZZY",  # Using fuzzy comparator
    threshold=0.7  # Explicit threshold for fuzzy matching
)
print(f"\nHUNGARIAN with FUZZY comparator (threshold 0.7):")
print(f"  Expected: {expected_list}")
print(f"  Actual: {actual_list}")
print(f"  Result: Matched={matched}, Score={score:.2f}")

# Test with NUMERIC comparator (non-matching case)
expected_num_list = ["$500", "$150.00", "$200"]
actual_num_list = ["500", "150", "210"]  # Different number representation for the last value
matched, score, reason = compare_values(
    expected=expected_num_list,
    actual=actual_num_list,
    method=EvaluationMethod.HUNGARIAN,
    comparator_type="NUMERIC"  # Using numeric comparator
)
print(f"\nHUNGARIAN with NUMERIC comparator (non-matching case):")
print(f"  Expected: {expected_num_list}")
print(f"  Actual: {actual_num_list}")
print(f"  Result: Matched={matched}, Score={score:.2f}")
print(f"  Note: The values '$200' and '210' don't match numerically")

# Test with NUMERIC comparator (matching case)
expected_num_list = ["$500", "$150.00", "$200"]
actual_num_list = ["500", "150", "200"]  # Exact numeric matches after normalization
matched, score, reason = compare_values(
    expected=expected_num_list,
    actual=actual_num_list,
    method=EvaluationMethod.HUNGARIAN,
    comparator_type="NUMERIC"  # Using numeric comparator
)
print(f"\nHUNGARIAN with NUMERIC comparator (matching case):")
print(f"  Expected: {expected_num_list}")
print(f"  Actual: {actual_num_list}")
print(f"  Result: Matched={matched}, Score={score:.2f}")
print(f"  Note: All numeric values match after normalization")

# HUNGARIAN method - Non-list values (should convert to list)
test_comparison(EvaluationMethod.HUNGARIAN, "Single item", "Single item")


------------------------------------------------------------
Method: HUNGARIAN
Expected: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $200']
Actual: ['Deposit: $500', 'Transfer: $200', 'Withdrawal: $150']
Matched: True
Score: 1.0

Demonstrating the new comparator_type functionality:
------------------------------------------------------------
HUNGARIAN with EXACT comparator:
  Expected: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $200']
  Actual: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $210']
  Result: Matched=False, Score=0.67

HUNGARIAN with FUZZY comparator (threshold 0.7):
  Expected: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $200']
  Actual: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $210']
  Result: Matched=True, Score=0.97

HUNGARIAN with NUMERIC comparator (non-matching case):
  Expected: ['$500', '$150.00', '$200']
  Actual: ['500', '150', '210']
  Result: Matched=False, Score=0.67
  Note: The values '$200' and '210' don't match numerically

HUN

(True, 1.0, None)

### Test 5: LLM Method
Testing semantic comparison using a Large Language Model.

In [8]:
# LLM method - High semantic match (different wording, same meaning)
test_comparison(
    EvaluationMethod.LLM,
    "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
    "Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.",
    document_class="BankStatement",
    attr_name="statement_summary",
    attr_description="Summary of the bank statement"
)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials



------------------------------------------------------------
Method: LLM
Expected: Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.
Actual: Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.


INFO:idp_common.bedrock:Bedrock request attempt 1/8:
INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '1a261044-c072-4dcc-b7f9-f159a4ba60a0', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 24 Apr 2025 15:24:33 GMT', 'content-type': 'application/json', 'content-length': '511', 'connection': 'keep-alive', 'x-amzn-requestid': '1a261044-c072-4dcc-b7f9-f159a4ba60a0'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n  "match": true,\n  "score": 0.95,\n  "reason": "The actual value conveys the same information as the expected value, despite slight differences in wording and structure. Both mention deposits of $1,250, withdrawals of $850, and a resulting balance of $2,400, which aligns with the expected summary."\n}\n```'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 245, 'outputTokens': 84, 'totalTokens': 329}, 'metrics': {'latencyMs': 581}}
INFO:idp_common.evaluation.comparator:LLM evaluation for statement

Matched: True
Score: 0.95
Reason: The actual value conveys the same information as the expected value, despite slight differences in wording and structure. Both mention deposits of $1,250, withdrawals of $850, and a resulting balance of $2,400, which aligns with the expected summary.


(True,
 0.95,
 'The actual value conveys the same information as the expected value, despite slight differences in wording and structure. Both mention deposits of $1,250, withdrawals of $850, and a resulting balance of $2,400, which aligns with the expected summary.')

In [9]:
# LLM method - No semantic match (different meaning)
test_comparison(
    EvaluationMethod.LLM,
    "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
    "Statement with deposits of $2,500 and withdrawals of $1,200, leaving a balance of $3,800.",
    document_class="BankStatement",
    attr_name="statement_summary",
    attr_description="Summary of the bank statement"
)

INFO:idp_common.bedrock:Bedrock request attempt 1/8:



------------------------------------------------------------
Method: LLM
Expected: Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.
Actual: Statement with deposits of $2,500 and withdrawals of $1,200, leaving a balance of $3,800.


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '94d553c0-5815-47cc-a1d3-de495f2dcfe5', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 24 Apr 2025 15:24:34 GMT', 'content-type': 'application/json', 'content-length': '411', 'connection': 'keep-alive', 'x-amzn-requestid': '94d553c0-5815-47cc-a1d3-de495f2dcfe5'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n  "match": false,\n  "score": 0.4,\n  "reason": "The expected and actual values describe different financial transactions and balances, despite some similarities in the type of information presented."\n}\n```'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 247, 'outputTokens': 48, 'totalTokens': 295}, 'metrics': {'latencyMs': 480}}
INFO:idp_common.evaluation.comparator:LLM evaluation for statement_summary (from code block): match=False, score=0.4, reason=The expected and actual values describe different financial transactions and balances, despite

Matched: False
Score: 0.4
Reason: The expected and actual values describe different financial transactions and balances, despite some similarities in the type of information presented.


(False,
 0.4,
 'The expected and actual values describe different financial transactions and balances, despite some similarities in the type of information presented.')

In [10]:
# LLM method - Partial semantic match (some differences)
test_comparison(
    EvaluationMethod.LLM,
    "Policy effective date: January 15, 2023 to January 14, 2024",
    "Policy period begins on Jan 15, 2023 and expires on Jan 15, 2024",
    document_class="InsurancePolicy",
    attr_name="policy_period",
    attr_description="The dates during which the insurance policy is effective"
)

INFO:idp_common.bedrock:Bedrock request attempt 1/8:



------------------------------------------------------------
Method: LLM
Expected: Policy effective date: January 15, 2023 to January 14, 2024
Actual: Policy period begins on Jan 15, 2023 and expires on Jan 15, 2024


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': 'b8d0c1ee-69c5-48ac-bacf-cfa069fbf2c6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 24 Apr 2025 15:24:36 GMT', 'content-type': 'application/json', 'content-length': '607', 'connection': 'keep-alive', 'x-amzn-requestid': 'b8d0c1ee-69c5-48ac-bacf-cfa069fbf2c6'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n  "match": true,\n  "score": 0.95,\n  "reason": "Both values convey the same information about the policy period, with slight differences in wording and formatting. The expected value specifies the period as \'January 15, 2023 to January 14, 2024\', while the actual value states \'begins on Jan 15, 2023 and expires on Jan 15, 2024\'. Despite these differences, the semantic meaning is equivalent."\n}\n```'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 245, 'outputTokens': 121, 'totalTokens': 366}, 'metrics': {'latencyMs': 835}}
INFO:idp_common.eva

Matched: True
Score: 0.95
Reason: Both values convey the same information about the policy period, with slight differences in wording and formatting. The expected value specifies the period as 'January 15, 2023 to January 14, 2024', while the actual value states 'begins on Jan 15, 2023 and expires on Jan 15, 2024'. Despite these differences, the semantic meaning is equivalent.


(True,
 0.95,
 "Both values convey the same information about the policy period, with slight differences in wording and formatting. The expected value specifies the period as 'January 15, 2023 to January 14, 2024', while the actual value states 'begins on Jan 15, 2023 and expires on Jan 15, 2024'. Despite these differences, the semantic meaning is equivalent.")

### Test 6: SEMANTIC Method
Testing semantic comparison using embeddings with different thresholds.

In [11]:
# SEMANTIC method - High similarity (different wording but same meaning)
test_comparison(
    EvaluationMethod.SEMANTIC,
    "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
    "Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.",
    threshold=0.8
)

# SEMANTIC method - Medium similarity (related content)
matched, score, _ = test_comparison(
    EvaluationMethod.SEMANTIC,
    "Policy effective date: January 15, 2023 to January 14, 2024",
    "Coverage period: From Jan 15, 2023 through January 15, 2024",
    threshold=0.8
)
print(f"With threshold=0.7: {score >= 0.7}")

# SEMANTIC method - Low similarity (different content)
test_comparison(
    EvaluationMethod.SEMANTIC,
    "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
    "Insurance policy with a premium of $850 per year and a deductible of $500.",
    threshold=0.8
)

# SEMANTIC method - Different threshold test
print("\nTesting different thresholds with medium similarity content:")
for threshold in [0.5, 0.7, 0.8, 0.9]:
    matched, score, _ = test_comparison(
        EvaluationMethod.SEMANTIC,
        "The patient was diagnosed with hypertension and prescribed lisinopril 10mg daily.",
        "Patient has high blood pressure and was given medication to take once per day.",
        threshold=threshold
    )
    print(f"Threshold {threshold}: Match={matched}, Score={score:.4f}")

INFO:idp_common.evaluation.comparator:Generating embeddings for semantic comparison using model: amazon.titan-embed-text-v1
INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:



------------------------------------------------------------
Method: SEMANTIC
Expected: Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.
Actual: Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.
Threshold: 0.8


INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:
INFO:idp_common.evaluation.comparator:Semantic similarity score: 0.9229
INFO:idp_common.evaluation.comparator:Generating embeddings for semantic comparison using model: amazon.titan-embed-text-v1
INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:


Matched: True
Score: 0.9229397546670253

------------------------------------------------------------
Method: SEMANTIC
Expected: Policy effective date: January 15, 2023 to January 14, 2024
Actual: Coverage period: From Jan 15, 2023 through January 15, 2024
Threshold: 0.8


INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:
INFO:idp_common.evaluation.comparator:Semantic similarity score: 0.4553
INFO:idp_common.evaluation.comparator:Generating embeddings for semantic comparison using model: amazon.titan-embed-text-v1
INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:


...

Matched: True
Score: 0.8105557996454206
Threshold 0.7: Match=True, Score=0.8106

------------------------------------------------------------
Method: SEMANTIC
Expected: The patient was diagnosed with hypertension and prescribed lisinopril 10mg daily.
Actual: Patient has high blood pressure and was given medication to take once per day.
Threshold: 0.8


INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:
INFO:idp_common.evaluation.comparator:Semantic similarity score: 0.8106
INFO:idp_common.evaluation.comparator:Generating embeddings for semantic comparison using model: amazon.titan-embed-text-v1
INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:


Matched: True
Score: 0.8105557996454206
Threshold 0.8: Match=True, Score=0.8106

------------------------------------------------------------
Method: SEMANTIC
Expected: The patient was diagnosed with hypertension and prescribed lisinopril 10mg daily.
Actual: Patient has high blood pressure and was given medication to take once per day.
Threshold: 0.9


INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:
INFO:idp_common.evaluation.comparator:Semantic similarity score: 0.8106


Matched: False
Score: 0.8105557996454206
Threshold 0.9: Match=False, Score=0.8106


### Comparing SEMANTIC vs LLM Methods

Let's compare results from the SEMANTIC method (using embeddings) with the LLM method on various examples to understand their respective strengths.

In [12]:
# Define a function to compare both methods
def compare_semantic_vs_llm(expected: str, actual: str, description: str = "Test comparison"):
    """Compare SEMANTIC and LLM methods on the same input."""
    print(f"\n{'-'*100}")
    print(f"Comparison: {description}")
    print(f"Expected: {expected}")
    print(f"Actual: {actual}")
    print(f"{'-'*100}")
    
    # Run semantic comparison
    semantic_matched, semantic_score, semantic_reason = test_comparison(
        EvaluationMethod.SEMANTIC,
        expected,
        actual,
        threshold=0.8
    )
    
    # Run LLM comparison
    llm_matched, llm_score, llm_reason = test_comparison(
        EvaluationMethod.LLM,
        expected,
        actual,
        document_class="TestDoc",
        attr_name="test_attr",
        attr_description="Test attribute"
    )
    
    # Compare results
    print("\nComparison of results:")
    print(f"{'Method':<10} {'Matched':<10} {'Score':<10} {'Reason'}")
    print(f"{'-'*80}")
    print(f"{'SEMANTIC':<10} {semantic_matched!s:<10} {semantic_score:<10.4f} {semantic_reason or ''}")
    print(f"{'LLM':<10} {llm_matched!s:<10} {llm_score:<10.4f} {llm_reason or ''}")
    
    return {
        "semantic": (semantic_matched, semantic_score, semantic_reason),
        "llm": (llm_matched, llm_score, llm_reason)
    }

# Test cases where both methods should give similar results
compare_semantic_vs_llm(
    "Patient diagnosed with pneumonia and prescribed antibiotics for 10 days.",
    "The patient has pneumonia and was given a 10-day course of antibiotics.",
    "Similar medical information with different wording"
)

# Test cases where SEMANTIC might be better
compare_semantic_vs_llm(
    "Total payment due: $1,543.27",
    "Amount to be paid: $1,543.27",
    "Financial information with different formats but exact amounts"
)

# Test cases where LLM might be better
compare_semantic_vs_llm(
    "Room temperature maintained at 72°F during the experiment.",
    "The experiment was conducted in standard laboratory conditions at room temperature.",
    "Implicit vs explicit information"
)

INFO:idp_common.evaluation.comparator:Generating embeddings for semantic comparison using model: amazon.titan-embed-text-v1
INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:



----------------------------------------------------------------------------------------------------
Comparison: Similar medical information with different wording
Expected: Patient diagnosed with pneumonia and prescribed antibiotics for 10 days.
Actual: The patient has pneumonia and was given a 10-day course of antibiotics.
----------------------------------------------------------------------------------------------------

------------------------------------------------------------
Method: SEMANTIC
Expected: Patient diagnosed with pneumonia and prescribed antibiotics for 10 days.
Actual: The patient has pneumonia and was given a 10-day course of antibiotics.
Threshold: 0.8


INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:
INFO:idp_common.evaluation.comparator:Semantic similarity score: 0.8057
INFO:idp_common.bedrock:Bedrock request attempt 1/8:


Matched: True
Score: 0.8056780651446329

------------------------------------------------------------
Method: LLM
Expected: Patient diagnosed with pneumonia and prescribed antibiotics for 10 days.
Actual: The patient has pneumonia and was given a 10-day course of antibiotics.


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': 'e80df115-688b-43d9-baa3-7f4d614cf08d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 24 Apr 2025 15:24:44 GMT', 'content-type': 'application/json', 'content-length': '455', 'connection': 'keep-alive', 'x-amzn-requestid': 'e80df115-688b-43d9-baa3-7f4d614cf08d'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n  "match": true,\n  "score": 0.95,\n  "reason": "Both values convey the same information: a patient diagnosed with pneumonia who was prescribed a 10-day course of antibiotics. The differences in wording and structure do not change the meaning."\n}\n```'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 214, 'outputTokens': 64, 'totalTokens': 278}, 'metrics': {'latencyMs': 654}}
INFO:idp_common.evaluation.comparator:LLM evaluation for test_attr (from code block): match=True, score=0.95, reason=Both values convey the same information: a patient diagnos

...

INFO:idp_common.bedrock:Bedrock embedding request attempt 1/8:
INFO:idp_common.evaluation.comparator:Semantic similarity score: 0.8604
INFO:idp_common.bedrock:Bedrock request attempt 1/8:


Matched: True
Score: 0.8603925642525886

------------------------------------------------------------
Method: LLM
Expected: Room temperature maintained at 72°F during the experiment.
Actual: The experiment was conducted in standard laboratory conditions at room temperature.


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '62124888-9690-4f9c-b887-00cc129744ff', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 24 Apr 2025 15:24:48 GMT', 'content-type': 'application/json', 'content-length': '447', 'connection': 'keep-alive', 'x-amzn-requestid': '62124888-9690-4f9c-b887-00cc129744ff'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n  "match": true,\n  "score": 0.95,\n  "reason": "Both the expected and actual values convey that the experiment was conducted at room temperature, which is semantically equivalent to \'72°F\' being a specific detail of room temperature."\n}\n```'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 209, 'outputTokens': 62, 'totalTokens': 271}, 'metrics': {'latencyMs': 648}}
INFO:idp_common.evaluation.comparator:LLM evaluation for test_attr (from code block): match=True, score=0.95, reason=Both the expected and actual values convey that the experiment wa

Matched: True
Score: 0.95
Reason: Both the expected and actual values convey that the experiment was conducted at room temperature, which is semantically equivalent to '72°F' being a specific detail of room temperature.

Comparison of results:
Method     Matched    Score      Reason
--------------------------------------------------------------------------------
SEMANTIC   True       0.8604     
LLM        True       0.9500     Both the expected and actual values convey that the experiment was conducted at room temperature, which is semantically equivalent to '72°F' being a specific detail of room temperature.


{'semantic': (True, 0.8603925642525886, None),
 'llm': (True,
  0.95,
  "Both the expected and actual values convey that the experiment was conducted at room temperature, which is semantically equivalent to '72°F' being a specific detail of room temperature.")}

In [13]:
# Setup test config
test_config = {
    "classes": [
        {
            "name": "TestDocument",
            "attributes": [
                {
                    "name": "exact_match_attr",
                    "description": "Attribute for exact matching",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "numeric_attr",
                    "description": "Attribute for numeric matching",
                    "evaluation_method": "NUMERIC_EXACT"
                },
                {
                    "name": "fuzzy_attr",
                    "description": "Attribute for fuzzy matching",
                    "evaluation_method": "FUZZY",
                    "evaluation_threshold": 0.8
                },
                {
                    "name": "list_attr",
                    "description": "Attribute for list comparison",
                    "evaluation_method": "HUNGARIAN",
                    "hungarian_comparator": "EXACT"
                },
                {
                    "name": "list_attr_fuzzy",
                    "description": "Attribute for list comparison with fuzzy matching",
                    "evaluation_method": "HUNGARIAN",
                    "hungarian_comparator": "FUZZY",
                    "evaluation_threshold": 0.7  # Threshold for fuzzy matching within Hungarian
                },
                {
                    "name": "list_attr_numeric",
                    "description": "Attribute for list comparison with numeric matching",
                    "evaluation_method": "HUNGARIAN",
                    "hungarian_comparator": "NUMERIC"
                },
                {
                    "name": "llm_attr",
                    "description": "Attribute for semantic comparison",
                    "evaluation_method": "LLM"
                },
                {
                    "name": "missing_in_actual",
                    "description": "Attribute missing in actual results",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "missing_in_expected",
                    "description": "Attribute missing in expected results",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "missing_everywhere",
                    "description": "Attribute missing in both expected and actual",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "semantic_attr",
                    "description": "Attribute for semantic embedding comparison",
                    "evaluation_method": "SEMANTIC",
                    "evaluation_threshold": 0.8
                }
            ]
        }
    ],
    "evaluation": {
        "llm_method": {
            "model": "us.amazon.nova-lite-v1:0",
            "temperature": 0.0,
            "top_k": 250,
            "system_prompt": "You are an evaluator for document extraction attributes.",
            "task_prompt": """I need to evaluate attribute extraction for a document of class: {DOCUMENT_CLASS}.

For the attribute named "{ATTRIBUTE_NAME}" described as "{ATTRIBUTE_DESCRIPTION}":
- Expected value: {EXPECTED_VALUE}
- Actual value: {ACTUAL_VALUE}

Do these values match in meaning, taking into account formatting differences, word order, abbreviations, and semantic equivalence?
Provide your assessment as a JSON with three fields:
- "match": boolean (true if they match, false if not)
- "score": number between 0 and 1 representing the confidence/similarity score
- "reason": brief explanation of your decision

Respond ONLY with the JSON and nothing else.  Here's the exact format:
{
  "match": true or false,
  "score": 0.0 to 1.0,
  "reason": "Your explanation here"
}
"""
        }
    }
}

In [14]:
# Update mock S3 retrieval function to include semantic_attr
def mock_s3_get_json(uri: str) -> Dict[str, Any]:
    """Mock S3 file retrieval."""
    if "expected" in uri:
        return {
            "exact_match_attr": "Exact Match Value",
            "numeric_attr": "$1,250.00",
            "fuzzy_attr": "John Alexander Smith",
            "list_attr": ["Item 1", "Item 2", "Item 3"],
            "list_attr_fuzzy": ["Payment method: Credit Card", "Due date: Jan 15, 2023", "Reference: ABC-123"],
            "list_attr_numeric": ["$500.00", "$150.00", "$200.00"],
            "llm_attr": "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
            "semantic_attr": "Patient was diagnosed with hypertension and prescribed lisinopril 10mg daily.",
            "missing_in_actual": "This value exists in expected only",
            # missing_in_expected is intentionally omitted
            # missing_everywhere is intentionally omitted
        }
    else:  # actual results
        return {
            "exact_match_attr": "Exact Match Value",  # Exact match
            "numeric_attr": 1250,  # Numeric match
            "fuzzy_attr": "John A Smith",  # Fuzzy match
            "list_attr": ["Item 1", "Item 3", "Item 2"],  # List with different order
            "list_attr_fuzzy": ["Reference: ABC123", "Payment: CC", "Due: January 15, 2023"],  # Fuzzy matches
            "list_attr_numeric": [500, 150, 200],  # Numeric comparison (now matches)
            "llm_attr": "Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.",  # Semantic match
            "semantic_attr": "Patient has high blood pressure and was given medication to take once per day.",  # Semantic embedding match
            # missing_in_actual is intentionally omitted
            "missing_in_expected": "This value exists in actual only",
            # missing_everywhere is intentionally omitted
        }

# Set up mock storage - we'll still use this for S3
class MockS3:
    # Store report content for later display
    report_content = ""
    results_content = {}
    
    @staticmethod
    def get_json_content(uri: str) -> Dict[str, Any]:
        return mock_s3_get_json(uri)
    
    @staticmethod
    def write_content(content: Any, bucket: str, key: str, content_type: str = None):
        print(f"Writing content to s3://{bucket}/{key}")
        if key.endswith("results.json"):
            # Store the results for later access
            MockS3.results_content = content
            print(f"Evaluation results summary: {json.dumps(content.get('overall_metrics', {}), indent=2)}")
        elif key.endswith("report.md"):
            # Store the markdown report for later display
            MockS3.report_content = content

In [15]:
# Create mock documents for evaluation
def create_test_document(doc_id: str, is_expected: bool = False) -> Document:
    """Create a test document with a section."""
    section = Section(
        section_id="sec-001",
        classification="TestDocument",
        extraction_result_uri=f"s3://test-bucket/{doc_id}/{'expected' if is_expected else 'actual'}/extraction.json"
    )
    
    doc = Document(
        id=doc_id,
        sections=[section],
        input_key=doc_id,
        input_bucket="test-bucket",
        output_bucket="test-bucket",
        status=Status.EXTRACTED
    )
    
    return doc

# Create test documents
actual_doc = create_test_document("test-doc-001")
expected_doc = create_test_document("test-doc-001-baseline", is_expected=True)

In [16]:
# Evaluate document
# Only patch S3 module - use real Bedrock
import idp_common.evaluation.service
idp_common.evaluation.service.s3 = MockS3

# Create evaluation service
evaluation_service = EvaluationService(region="us-east-1", config=test_config)

# Evaluate document
result_doc = evaluation_service.evaluate_document(actual_doc, expected_doc, store_results=True)

# Print results
if hasattr(result_doc, 'evaluation_result'):
    eval_result = result_doc.evaluation_result
    print(f"\nOverall metrics: {eval_result.overall_metrics}")
    
    # Check section results
    for section_result in eval_result.section_results:
        print(f"\nSection {section_result.section_id} - Class: {section_result.document_class}")
        print(f"Metrics: {section_result.metrics}")
        
        # Print attribute details
        print("\nAttribute Details:")
        print("-" * 100)
        print(f"{'Name':<20} {'Method':<15} {'Expected':<25} {'Actual':<25} {'Matched':<10} {'Score':<10} {'Reason'}")
        print("-" * 100)
        
        for attr in section_result.attributes:
            expected_val = str(attr.expected)[:25]
            actual_val = str(attr.actual)[:25]
            method = attr.evaluation_method
            reason = attr.reason[:50] + "..." if attr.reason and len(attr.reason) > 50 else (attr.reason or "")
            print(f"{attr.name:<20} {method:<15} {expected_val:<25} {actual_val:<25} {attr.matched!s:<10} {attr.score:<10.2f} {reason}")

INFO:idp_common.evaluation.service:Initialized evaluation service with LLM configuration and max_workers=10
INFO:idp_common.evaluation.service:Comparing: exact_match_attr using EvaluationMethod.EXACT - from class TestDocument
INFO:idp_common.evaluation.service:Comparing: numeric_attr using EvaluationMethod.NUMERIC_EXACT - from class TestDocument
INFO:idp_common.evaluation.service:Comparing: fuzzy_attr using EvaluationMethod.FUZZY - from class TestDocument
INFO:idp_common.evaluation.service:Comparing: list_attr using EvaluationMethod.HUNGARIAN - from class TestDocument
INFO:idp_common.evaluation.service:Comparing: list_attr_fuzzy using EvaluationMethod.HUNGARIAN - from class TestDocument
INFO:idp_common.evaluation.service:Comparing: list_attr_numeric using EvaluationMethod.HUNGARIAN - from class TestDocument
INFO:idp_common.evaluation.service:Comparing: missing_in_actual using EvaluationMethod.EXACT - from class TestDocument
INFO:idp_common.evaluation.service:Comparing: missing_in_expec

Writing content to s3://test-bucket/test-doc-001/evaluation/results.json
Evaluation results summary: {
  "precision": 0.5555555555555556,
  "recall": 0.8333333333333334,
  "f1_score": 0.6666666666666667,
  "accuracy": 0.5454545454545454,
  "false_alarm_rate": 0.5,
  "false_discovery_rate": 0.375
}
Writing content to s3://test-bucket/test-doc-001/evaluation/report.md

Overall metrics: {'precision': 0.5555555555555556, 'recall': 0.8333333333333334, 'f1_score': 0.6666666666666667, 'accuracy': 0.5454545454545454, 'false_alarm_rate': 0.5, 'false_discovery_rate': 0.375}

Section sec-001 - Class: TestDocument
Metrics: {'precision': 0.5555555555555556, 'recall': 0.8333333333333334, 'f1_score': 0.6666666666666667, 'accuracy': 0.5454545454545454, 'false_alarm_rate': 0.5, 'false_discovery_rate': 0.375}

Attribute Details:
----------------------------------------------------------------------------------------------------
Name                 Method          Expected                  Actual       

### Display the Evaluation Report

Let's display the markdown evaluation report that was generated:

In [17]:
from IPython.display import Markdown

# Display the markdown report
if MockS3.report_content:
    display(Markdown(MockS3.report_content))
else:
    print("No evaluation report was generated.")

# Document Evaluation: test-doc-001

## Summary
- **Match Rate**: 🟠 6/11 attributes matched [██████████░░░░░░░░░░] 54%
- **Precision**: 0.56 | **Recall**: 0.83 | **F1 Score**: 🟠 0.67

## Overall Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.5556 | 🟠 Fair |
| recall | 0.8333 | 🟡 Good |
| f1_score | 0.6667 | 🟠 Fair |
| accuracy | 0.5455 | 🟠 Fair |
| false_alarm_rate | 0.5000 | 🟠 Fair |
| false_discovery_rate | 0.3750 | 🟠 Fair |


## Section: sec-001 (TestDocument)
### Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.5556 | 🟠 Fair |
| recall | 0.8333 | 🟡 Good |
| f1_score | 0.6667 | 🟠 Fair |
| accuracy | 0.5455 | 🟠 Fair |
| false_alarm_rate | 0.5000 | 🟠 Fair |
| false_discovery_rate | 0.3750 | 🟠 Fair |


### Attributes
| Status | Attribute | Expected | Actual | Score | Method | Reason |
| :----: | --------- | -------- | ------ | ----- | ------ | ------ |
| ✅ | exact_match_attr | Exact Match Value | Exact Match Value | 1.00 | EXACT |  |
| ❌ | fuzzy_attr | John Alexander Smith | John A Smith | 0.60 | FUZZY (threshold: 0.8) |  |
| ✅ | list_attr | ['Item 1', 'Item 2', 'Item 3'] | ['Item 1', 'Item 3', 'Item 2'] | 1.00 | HUNGARIAN (comparator: EXACT) |  |
| ❌ | list_attr_fuzzy | ['Payment method: Credit Card', 'Due date: Jan 15, 2023', 'Reference: ABC-123'] | ['Reference: ABC123', 'Payment: CC', 'Due: January 15, 2023'] | 0.68 | HUNGARIAN (comparator: FUZZY, threshold: 0.7) |  |
| ✅ | list_attr_numeric | ['$500.00', '$150.00', '$200.00'] | [500, 150, 200] | 1.00 | HUNGARIAN (comparator: NUMERIC) |  |
| ✅ | llm_attr | Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400. | Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400. | 0.95 | LLM | Both values describe the same financial transactions and resulting balance, despite slight differences in wording and structure. The semantic meaning is preserved. |
| ✅ | missing_everywhere | None | None | 1.00 | EXACT | Both actual and expected values are missing, so they are matched. |
| ❌ | missing_in_actual | This value exists in expected only | None | 0.00 | EXACT |  |
| ❌ | missing_in_expected | None | This value exists in actual only | 0.00 | EXACT |  |
| ✅ | numeric_attr | $1,250.00 | 1250 | 1.00 | NUMERIC_EXACT |  |
| ❌ | semantic_attr | Patient was diagnosed with hypertension and prescribed lisinopril 10mg daily. | Patient has high blood pressure and was given medication to take once per day. | 0.80 | SEMANTIC (threshold: 0.8) |  |


Execution time: 1.48 seconds

## Evaluation Methods Used

This evaluation used the following methods to compare expected and actual values:

1. **EXACT** - Exact string match after stripping punctuation and whitespace
2. **NUMERIC_EXACT** - Exact numeric match after normalizing
3. **FUZZY** - Fuzzy string matching using string similarity metrics (with evaluation_threshold)
4. **SEMANTIC** - Semantic similarity comparison using Bedrock Titan embeddings (with evaluation_threshold)
5. **HUNGARIAN** - Bipartite matching algorithm for lists of values
   - **EXACT** - Hungarian matching with exact string comparison
   - **FUZZY** - Hungarian matching with fuzzy string comparison (with evaluation_threshold)
   - **NUMERIC** - Hungarian matching with numeric comparison
6. **LLM** - Advanced semantic evaluation using Bedrock large language models

Each attribute is configured with a specific evaluation method based on the data type and comparison needs.

## Part 3: Smart Attribute Discovery and Evaluation

This section demonstrates the "Smart attribute discovery and evaluation" feature of the EvaluationService, which:
1. Automatically discovers attributes in the data not defined in configuration
2. Applies default evaluation methods to unconfigured attributes
3. Properly handles attributes found only in expected data, only in actual data, or in both

In [18]:
# Define a minimal configuration with only some attributes defined
minimal_config = {
    "classes": [
        {
            "name": "InvoiceDocument",
            "attributes": [
                {
                    "name": "invoice_number",
                    "description": "The unique identifier for the invoice",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "amount_due",
                    "description": "The total amount to be paid",
                    "evaluation_method": "NUMERIC_EXACT"
                }
                # Note: Other attributes are intentionally omitted from configuration
            ]
        }
    ],
    "evaluation": {
        "llm_method": {
            "model": "us.amazon.nova-lite-v1:0",
            "temperature": 0.0,
            "top_k": 250,
            "system_prompt": "You are an evaluator for document extraction attributes.",
            "task_prompt": """I need to evaluate attribute extraction for a document of class: {DOCUMENT_CLASS}.

For the attribute named "{ATTRIBUTE_NAME}" described as "{ATTRIBUTE_DESCRIPTION}":
- Expected value: {EXPECTED_VALUE}
- Actual value: {ACTUAL_VALUE}

Do these values match in meaning, taking into account formatting differences, word order, abbreviations, and semantic equivalence?
Provide your assessment as a JSON with three fields:
- "match": boolean (true if they match, false if not)
- "score": number between 0 and 1 representing the confidence/similarity score
- "reason": brief explanation of your decision

Respond ONLY with the JSON and nothing else.  Here's the exact format:
{
  "match": true or false,
  "score": 0.0 to 1.0,
  "reason": "Your explanation here"
}
"""
        }
    }
}

In [19]:
# Override S3 mock data function to simulate invoice data with unconfigured attributes
def mock_invoice_s3_get_json(uri: str) -> Dict[str, Any]:
    """Mock S3 file retrieval with invoice data including unconfigured attributes."""
    if "expected" in uri:
        return {
            # Configured attributes
            "invoice_number": "INV-12345",  # Configured with EXACT method
            "amount_due": "$1,250.00",     # Configured with NUMERIC_EXACT method
            
            # Unconfigured attributes - only in expected
            "reference_number": "REF-98765",
            
            # Unconfigured attributes - in both expected and actual
            "issue_date": "January 15, 2023",
            "due_date": "February 15, 2023",
            "vendor_name": "Acme Corporation Inc.",
            "payment_terms": "Net 30"
        }
    else:  # actual results
        return {
            # Configured attributes
            "invoice_number": "INV-12345",  # Exact match with expected
            "amount_due": 1250,            # Numeric match with expected
            
            # Unconfigured attributes - only in actual
            "purchase_order": "PO-54321",
            
            # Unconfigured attributes - in both expected and actual with various match qualities
            "issue_date": "01/15/2023",                     # Different format but same date
            "due_date": "02/15/2023",                       # Different format but same date
            "vendor_name": "ACME Corp.",                    # Abbreviated but similar
            "payment_terms": "Payment due within 30 days"    # Different wording but same meaning
        }

# Create test invoice documents
def create_invoice_document(doc_id: str, is_expected: bool = False) -> Document:
    """Create a test invoice document with a section."""
    section = Section(
        section_id="inv-001",
        classification="InvoiceDocument",
        extraction_result_uri=f"s3://test-bucket/{doc_id}/{'expected' if is_expected else 'actual'}/extraction.json"
    )
    
    doc = Document(
        id=doc_id,
        sections=[section],
        input_key=doc_id,
        input_bucket="test-bucket",
        output_bucket="test-bucket",
        status=Status.EXTRACTED
    )
    
    return doc

# Set up a new mock for this example
class MockInvoiceS3:
    # Store report content for later display
    report_content = ""
    results_content = {}
    
    @staticmethod
    def get_json_content(uri: str) -> Dict[str, Any]:
        return mock_invoice_s3_get_json(uri)
    
    @staticmethod
    def write_content(content: Any, bucket: str, key: str, content_type: str = None):
        print(f"Writing content to s3://{bucket}/{key}")
        if key.endswith("results.json"):
            # Store the results for later access
            MockInvoiceS3.results_content = content
        elif key.endswith("report.md"):
            # Store the markdown report for later display
            MockInvoiceS3.report_content = content

In [20]:
# Evaluate document with smart attribute discovery
# Patch S3 module with our invoice mock
idp_common.evaluation.service.s3 = MockInvoiceS3

# Create test documents
actual_invoice = create_invoice_document("invoice-001")
expected_invoice = create_invoice_document("invoice-001-baseline", is_expected=True)

# Create evaluation service with minimal config
evaluation_service = EvaluationService(region="us-east-1", config=minimal_config)

# Evaluate document
print("Evaluating invoice document with smart attribute discovery...")
result_doc = evaluation_service.evaluate_document(actual_invoice, expected_invoice, store_results=True)

# Print results with focus on discovered attributes
if hasattr(result_doc, 'evaluation_result'):
    eval_result = result_doc.evaluation_result
    print(f"\nOverall metrics: {eval_result.overall_metrics}")
    
    # Check section results
    for section_result in eval_result.section_results:
        print(f"\nSection {section_result.section_id} - Class: {section_result.document_class}")
        
        # Find configured vs unconfigured attributes
        configured_attrs = []
        unconfigured_attrs = []
        
        for attr in section_result.attributes:
            # Check if the attribute has a message about being unconfigured
            if attr.reason and "attribute not in the configuration" in attr.reason:
                unconfigured_attrs.append(attr)
            else:
                configured_attrs.append(attr)
        
        # Print summary of attribute counts
        print(f"Total attributes evaluated: {len(section_result.attributes)}")
        print(f"  - Configured attributes: {len(configured_attrs)}")
        print(f"  - Auto-discovered attributes: {len(unconfigured_attrs)}")
        
        # Print attribute details - CONFIGURED
        if configured_attrs:
            print("\nCONFIGURED Attribute Details:")
            print("-" * 100)
            print(f"{'Name':<20} {'Method':<15} {'Expected':<25} {'Actual':<25} {'Matched':<10} {'Score':<10}")
            print("-" * 100)
            
            for attr in configured_attrs:
                expected_val = str(attr.expected)[:25]
                actual_val = str(attr.actual)[:25]
                method = attr.evaluation_method
                print(f"{attr.name:<20} {method:<15} {expected_val:<25} {actual_val:<25} {attr.matched!s:<10} {attr.score:<10.2f}")
        
        # Print attribute details - UNCONFIGURED
        if unconfigured_attrs:
            print("\nAUTO-DISCOVERED Attribute Details:")
            print("-" * 100)
            print(f"{'Name':<20} {'Method':<15} {'Expected':<25} {'Actual':<25} {'Matched':<10} {'Score':<10} {'Reason'}")
            print("-" * 100)
            
            for attr in unconfigured_attrs:
                expected_val = str(attr.expected)[:25]
                actual_val = str(attr.actual)[:25]
                method = attr.evaluation_method
                reason = attr.reason[:50] + "..." if attr.reason and len(attr.reason) > 50 else (attr.reason or "")
                print(f"{attr.name:<20} {method:<15} {expected_val:<25} {actual_val:<25} {attr.matched!s:<10} {attr.score:<10.2f} {reason}")

INFO:idp_common.evaluation.service:Initialized evaluation service with LLM configuration and max_workers=10
INFO:idp_common.evaluation.service:Comparing: invoice_number using EvaluationMethod.EXACT - from class InvoiceDocument
INFO:idp_common.evaluation.service:Comparing: amount_due using EvaluationMethod.NUMERIC_EXACT - from class InvoiceDocument
INFO:idp_common.evaluation.service:Comparing: reference_number using EvaluationMethod.LLM - from class InvoiceDocument
INFO:idp_common.evaluation.service:Comparing: purchase_order using EvaluationMethod.LLM - from class InvoiceDocument
INFO:idp_common.evaluation.service:Comparing: vendor_name using EvaluationMethod.LLM - from class InvoiceDocument
INFO:idp_common.evaluation.service:Comparing: payment_terms using EvaluationMethod.LLM - from class InvoiceDocument
INFO:idp_common.evaluation.service:Comparing: due_date using EvaluationMethod.LLM - from class InvoiceDocument
INFO:idp_common.evaluation.service:Comparing: issue_date using Evaluation

Evaluating invoice document with smart attribute discovery...


INFO:idp_common.bedrock:Bedrock request attempt 1/8:
INFO:idp_common.bedrock:Bedrock request attempt 1/8:
INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '80859633-7665-4d07-8b2a-a6d91a395bd1', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 24 Apr 2025 15:24:51 GMT', 'content-type': 'application/json', 'content-length': '518', 'connection': 'keep-alive', 'x-amzn-requestid': '80859633-7665-4d07-8b2a-a6d91a395bd1'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '```json\n{\n  "match": true,\n  "score": 0.95,\n  "reason": "The expected value \'Acme Corporation Inc.\' and the actual value \'ACME Corp.\' refer to the same entity despite differences in formatting and abbreviations. \'Acme\' and \'ACME\' are case variations, \'Corporation\' is abbreviated to \'Corp.\', and \'Inc.\' is omitted."\n}\n```'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 188, 'outputTokens': 90, 'totalTokens': 278}, 'metrics': {'latencyMs'

Writing content to s3://test-bucket/invoice-001/evaluation/results.json
Writing content to s3://test-bucket/invoice-001/evaluation/report.md

Overall metrics: {'precision': 0.8571428571428571, 'recall': 0.8571428571428571, 'f1_score': 0.8571428571428571, 'accuracy': 0.75, 'false_alarm_rate': 1.0, 'false_discovery_rate': 0.0}

Section inv-001 - Class: InvoiceDocument
Total attributes evaluated: 8
  - Configured attributes: 8
  - Auto-discovered attributes: 0

CONFIGURED Attribute Details:
----------------------------------------------------------------------------------------------------
Name                 Method          Expected                  Actual                    Matched    Score     
----------------------------------------------------------------------------------------------------
amount_due           NUMERIC_EXACT   $1,250.00                 1250                      True       1.00      
due_date             LLM             February 15, 2023         02/15/2023          

### Display the Smart Attribute Discovery Evaluation Report

Let's display the markdown evaluation report that was generated for the smart attribute discovery scenario:

In [21]:
# Display the markdown report
if MockInvoiceS3.report_content:
    display(Markdown(MockInvoiceS3.report_content))
else:
    print("No evaluation report was generated.")

# Document Evaluation: invoice-001

## Summary
- **Match Rate**: 🟡 6/8 attributes matched [███████████████░░░░░] 75%
- **Precision**: 0.86 | **Recall**: 0.86 | **F1 Score**: 🟡 0.86

## Overall Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.8571 | 🟡 Good |
| recall | 0.8571 | 🟡 Good |
| f1_score | 0.8571 | 🟡 Good |
| accuracy | 0.7500 | 🟡 Good |
| false_alarm_rate | 1.0000 | 🔴 Poor |
| false_discovery_rate | 0.0000 | 🟢 Excellent |


## Section: inv-001 (InvoiceDocument)
### Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.8571 | 🟡 Good |
| recall | 0.8571 | 🟡 Good |
| f1_score | 0.8571 | 🟡 Good |
| accuracy | 0.7500 | 🟡 Good |
| false_alarm_rate | 1.0000 | 🔴 Poor |
| false_discovery_rate | 0.0000 | 🟢 Excellent |


### Attributes
| Status | Attribute | Expected | Actual | Score | Method | Reason |
| :----: | --------- | -------- | ------ | ----- | ------ | ------ |
| ✅ | amount_due | $1,250.00 | 1250 | 1.00 | NUMERIC_EXACT |  |
| ✅ | due_date | February 15, 2023 | 02/15/2023 | 0.95 | LLM | The expected value 'February 15, 2023' and the actual value '02/15/2023' represent the same date despite differences in formatting. Both values convey the same semantic information. [Default method - attribute not specified in the configuration] |
| ✅ | invoice_number | INV-12345 | INV-12345 | 1.00 | EXACT |  |
| ✅ | issue_date | January 15, 2023 | 01/15/2023 | 0.95 | LLM | Both values represent the same date, January 15, 2023, despite differences in formatting. The expected value is in a word format while the actual value is in a numeric format, but both clearly convey the same date. [Default method - attribute not specified in the configuration] |
| ✅ | payment_terms | Net 30 | Payment due within 30 days | 0.95 | LLM | Both values convey the same payment terms, with 'Net 30' being a common abbreviation for 'Payment due within 30 days'. Despite slight differences in wording and formatting, the semantic meaning is equivalent. [Default method - attribute not specified in the configuration] |
| ❌ | purchase_order | None | PO-54321 | 0.00 | LLM | [Default method - attribute not specified in the configuration] |
| ❌ | reference_number | REF-98765 | None | 0.00 | LLM | [Default method - attribute not specified in the configuration] |
| ✅ | vendor_name | Acme Corporation Inc. | ACME Corp. | 0.95 | LLM | The expected value 'Acme Corporation Inc.' and the actual value 'ACME Corp.' refer to the same entity despite differences in formatting and abbreviations. 'Acme' and 'ACME' are case variations, 'Corporation' is abbreviated to 'Corp.', and 'Inc.' is omitted. [Default method - attribute not specified in the configuration] |


Execution time: 3.10 seconds

## Evaluation Methods Used

This evaluation used the following methods to compare expected and actual values:

1. **EXACT** - Exact string match after stripping punctuation and whitespace
2. **NUMERIC_EXACT** - Exact numeric match after normalizing
3. **FUZZY** - Fuzzy string matching using string similarity metrics (with evaluation_threshold)
4. **SEMANTIC** - Semantic similarity comparison using Bedrock Titan embeddings (with evaluation_threshold)
5. **HUNGARIAN** - Bipartite matching algorithm for lists of values
   - **EXACT** - Hungarian matching with exact string comparison
   - **FUZZY** - Hungarian matching with fuzzy string comparison (with evaluation_threshold)
   - **NUMERIC** - Hungarian matching with numeric comparison
6. **LLM** - Advanced semantic evaluation using Bedrock large language models

Each attribute is configured with a specific evaluation method based on the data type and comparison needs.

### Smart Attribute Discovery Scenario Summary

The smart attribute discovery and evaluation feature provides the following benefits:

1. **Auto-discovery of attributes**
   - Finds attributes not explicitly defined in the configuration
   - Compares all data fields across expected and actual results
   - Works with minimal or even no attribute configuration

2. **Default Evaluation Method**
   - Applies LLM method to unconfigured attributes by default 
   - Provides semantic comparison for discovered attributes
   - Attaches explanations that the attribute was not in configuration

3. **Handles All Possible Cases**
   - Attributes in both expected and actual results
   - Attributes only in expected results (false negatives)
   - Attributes only in actual results (false positives)
   - Attributes that don't exist in either (true negatives)

4. **Benefits**
   - Exploratory evaluation without complete configuration
   - Comprehensive metrics that include all found attributes
   - Flexibility as extraction models evolve or change output formats
   - Identification of potential new attributes to add to configuration

This feature is particularly useful during the early stages of implementation when the complete attribute schema may not be fully defined, or when handling variations in extraction outputs that contain unexpected information.

<cell_type>markdown</cell_type>## Summary of All Demonstrated Features

This notebook has demonstrated:

1. All evaluation methods available in the IDP library:
   - EXACT - Exact string matching
   - NUMERIC_EXACT - Numeric value matching
   - FUZZY - Fuzzy string matching with adjustable thresholds
   - SEMANTIC - Semantic similarity comparison using Titan embeddings
   - HUNGARIAN - List comparison using the Hungarian algorithm with configurable comparator types:
     - EXACT comparator - Exact string matching for list items
     - FUZZY comparator - Fuzzy string matching for list items
     - NUMERIC comparator - Numeric comparison for list items
   - LLM - Semantic comparison using Large Language Models

2. Semantic Comparison Methods:
   - SEMANTIC - Uses Bedrock Titan embeddings and cosine similarity for efficient matching
   - LLM - Uses Bedrock Claude for more nuanced semantic understanding with reasoning

3. Benefits of SEMANTIC vs LLM methods:
   - SEMANTIC is faster and less expensive than LLM-based evaluation
   - LLM provides explanations for matches/mismatches
   - SEMANTIC works well for standard text comparisons
   - LLM better understands implicit information and complex reasoning

4. Handling of edge cases:
   - Attributes missing in actual results
   - Attributes missing in expected results
   - Attributes missing in both actual and expected results
   - Empty string values

5. Full document evaluation with mixed evaluation methods
   - Comprehensive metrics calculation
   - Detailed attribute-level results

6. Threshold sensitivity analysis for fuzzy and semantic matching
   - How different threshold values affect match results
   - Trade-offs between precision and recall

7. Smart attribute discovery and evaluation:
   - Auto-discovery of attributes not in configuration
   - Default semantic evaluation with LLM method
   - Comprehensive handling of all attribute cases
   - Support for exploratory evaluation and evolving schemas

## Summary

This notebook has demonstrated:

1. All evaluation methods available in the IDP library:
   - EXACT - Exact string matching
   - NUMERIC_EXACT - Numeric value matching
   - FUZZY - Fuzzy string matching with adjustable thresholds
   - SEMANTIC - Semantic similarity comparison using Titan embeddings
   - HUNGARIAN - List comparison using the Hungarian algorithm
   - LLM - Semantic comparison using Large Language Models

2. Semantic Comparison Methods:
   - SEMANTIC - Uses Bedrock Titan embeddings and cosine similarity for efficient matching
   - LLM - Uses Bedrock Claude for more nuanced semantic understanding with reasoning

3. Benefits of SEMANTIC vs LLM methods:
   - SEMANTIC is faster and less expensive than LLM-based evaluation
   - LLM provides explanations for matches/mismatches
   - SEMANTIC works well for standard text comparisons
   - LLM better understands implicit information and complex reasoning

4. Handling of edge cases:
   - Attributes missing in actual results
   - Attributes missing in expected results
   - Attributes missing in both actual and expected results
   - Empty string values

5. Full document evaluation with mixed evaluation methods
   - Comprehensive metrics calculation
   - Detailed attribute-level results

6. Threshold sensitivity analysis for fuzzy and semantic matching
   - How different threshold values affect match results
   - Trade-offs between precision and recall